In [2]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from random import randint
from collections import OrderedDict 
import numpy as np
import pandas as pd
import random
import sys
import io
import re
from unidecode import unidecode

In [3]:
songs = pd.read_csv('data/travis-scott-lyrics.csv')

In [4]:
def get_tokenized_lines(df):
    words = []
    
    for index, row in df['lyrics'].iteritems():
        row = str(row).lower()
        for line in row.split('|-|'):
            new_words = re.findall(r"\b[a-z']+\b", unidecode(line))
            words = words + new_words
        
    return words

In [5]:
all_lyric_lines = get_tokenized_lines(songs)

In [6]:
SEQ_LENGTH = 50 + 1
sequences = list()

for i in range(SEQ_LENGTH, len(all_lyric_lines)):
    seq = all_lyric_lines[i - SEQ_LENGTH: i]
    sequences.append(seq)

print('Total Sequences: %d' % len(sequences))

Total Sequences: 15330


In [7]:
def save_doc(lines, filename):
    for line in lines:
        data = ' '.join(line)
        
    '\n'.join(data)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [8]:
out_filename = 'sequences.txt'
save_doc(sequences, out_filename)

In [9]:
vocab = set(all_lyric_lines)
vocab = sorted(vocab)

word_to_index = {w: i for i, w in enumerate(vocab)}
index_to_word = {i: w for w, i in word_to_index.items()}
word_indices = [word_to_index[word] for word in vocab]
vocab_size = len(vocab)

print(word_to_index)
print('vocabulary size: {}'.format(vocab_size))

{"'s": 0, 'a': 1, 'aaaw': 2, 'about': 3, 'absolute': 4, 'accountant': 5, 'act': 6, 'actavis': 7, 'acting': 8, 'addiction': 9, 'address': 10, 'addressing': 11, 'adidas': 12, 'afford': 13, 'after': 14, 'again': 15, 'ages': 16, 'ago': 17, 'agree': 18, 'ahead': 19, 'ahh': 20, 'ahhh': 21, 'ahhhh': 22, "ain't": 23, 'akon': 24, 'album': 25, 'alcohol': 26, 'alert': 27, 'alive': 28, 'all': 29, 'almost': 30, 'alone': 31, 'already': 32, 'alright': 33, 'always': 34, 'am': 35, 'amazing': 36, 'amen': 37, 'amenities': 38, 'an': 39, 'and': 40, 'angel': 41, 'angels': 42, 'angles': 43, 'anna': 44, 'another': 45, 'answer': 46, 'anthem': 47, 'anti': 48, 'antidote': 49, 'any': 50, 'anything': 51, 'aod': 52, 'ap': 53, 'apparently': 54, 'appetite': 55, 'applebees': 56, 'are': 57, 'argue': 58, 'argument': 59, 'around': 60, 'artists': 61, 'as': 62, 'ask': 63, 'asked': 64, 'askin': 65, 'asleep': 66, 'ass': 67, 'astro': 68, 'astronomical': 69, 'astroworld': 70, 'at': 71, 'atlanta': 72, 'automatic': 73, 'avenue':

In [10]:
def get_tokenized_lines(lines, seq_len):
    tokenized = np.zeros((len(lines), seq_len))
    
    for r, line in enumerate(lines):
        for c, word in enumerate(line):
            tokenized[r, c] = word_to_index[word]

    return tokenized

In [11]:
tokenized_seq = get_tokenized_lines(sequences, SEQ_LENGTH)

In [12]:
tokenized_seq[:, -1].shape

(15330,)

In [13]:
from keras.utils import to_categorical

X, y = tokenized_seq[:, :-1], tokenized_seq[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = len(X[0])

In [14]:
print("X_shape", X.shape)
print("y_shape", y.shape)

X_shape (15330, 50)
y_shape (15330, 2054)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, batch_size=128, epochs=200)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            102700    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 2054)              207454    
Total params: 461,054
Trainable params: 461,054
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
120/120 [==============================] - 13s 110ms/step - loss: 6.4214 - accuracy: 0.0353
Epoch 2

120/120 [==============================] - 15s 129ms/step - loss: 2.4160 - accuracy: 0.4427
Epoch 70/200
120/120 [==============================] - 16s 130ms/step - loss: 2.3875 - accuracy: 0.4494
Epoch 71/200
120/120 [==============================] - 15s 127ms/step - loss: 2.3571 - accuracy: 0.4578
Epoch 72/200
120/120 [==============================] - 17s 144ms/step - loss: 2.3299 - accuracy: 0.4624
Epoch 73/200
120/120 [==============================] - 16s 137ms/step - loss: 2.3052 - accuracy: 0.4684
Epoch 74/200
120/120 [==============================] - 16s 133ms/step - loss: 2.2770 - accuracy: 0.4744
Epoch 75/200
120/120 [==============================] - 16s 131ms/step - loss: 2.2550 - accuracy: 0.4789
Epoch 76/200
120/120 [==============================] - 17s 144ms/step - loss: 2.2372 - accuracy: 0.4798
Epoch 77/200
120/120 [==============================] - 16s 131ms/step - loss: 2.2107 - accuracy: 0.4876
Epoch 78/200
120/120 [==============================] - 20s 168ms/st

120/120 [==============================] - 13s 106ms/step - loss: 1.1215 - accuracy: 0.7294
Epoch 147/200
120/120 [==============================] - 12s 104ms/step - loss: 1.1109 - accuracy: 0.7284
Epoch 148/200
120/120 [==============================] - 12s 102ms/step - loss: 1.1067 - accuracy: 0.7321
Epoch 149/200
120/120 [==============================] - 12s 103ms/step - loss: 1.0912 - accuracy: 0.7362
Epoch 150/200
120/120 [==============================] - 12s 103ms/step - loss: 1.0812 - accuracy: 0.7395
Epoch 151/200
120/120 [==============================] - 16s 134ms/step - loss: 1.0743 - accuracy: 0.7391
Epoch 152/200
120/120 [==============================] - 17s 143ms/step - loss: 1.0564 - accuracy: 0.7431
Epoch 153/200
120/120 [==============================] - 15s 126ms/step - loss: 1.0591 - accuracy: 0.7416
Epoch 154/200
120/120 [==============================] - 13s 111ms/step - loss: 1.0516 - accuracy: 0.7412
Epoch 155/200
120/120 [==============================] - 13s

In [ ]:
model.save('word_model_eminem.h5')

In [56]:
from keras.models import load_model

model = load_model('word_model_travis_scott.h5')

In [57]:
def texts_to_sequences(texts, word_to_index):
    indices = np.zeros((1, len(texts)), dtype=int)
    
    for i, text in enumerate(texts):
        if text not in word_to_index:
            random = index_to_word[randint(0,vocab_size)]
            indices[:, i] = word_to_index[random]
        else:
            indices[:, i] = word_to_index[text]
        
    return indices

In [58]:
def my_pad_sequences(seq, maxlen):
    start = seq.shape[1] - maxlen
    
    return seq[:, start: start + maxlen]

In [59]:
def generate_seq(model, word_to_index, seq_length, n_words):
    generated = ''
    usr_input = input("Write the beginning of your poem, the Drake machine will complete it. Your input is: ")
    generated += usr_input 
    sys.stdout.write("\n\nHere is your poem: \n\n") 
    
    result = list()
    in_text = [None] * 51
    generated_list = generated.split()
    
    # if input is shorter than 51 words, fill the beginning with random words
    if(len(generated_list) < 51):
        end = len(generated_list)
        for i in range (51 - end):
            random = index_to_word[randint(0,vocab_size)]
            in_text[i] = random
            
        index = 0
        for i in range (51 - end, 51):
            in_text[i] = generated_list[index]
            index += 1

    # if input is longer than 51 words, only use the last 51 words
    if(len(generated_list) > 51):
        end = len(generated_list)
        in_text = generated_list[end-51:]

    # generate words based on last 50 words
    for _ in range(n_words):
        encoded = texts_to_sequences(in_text[1:], word_to_index)
        encoded = my_pad_sequences(encoded, maxlen=seq_length)
        
        yhat = model.predict_classes(encoded, verbose=0)
        out_word = ''
    
        for word, index in word_to_index.items():
            if index == yhat:
                out_word = word
                break
        
        in_text += ' ' + out_word
        result.append(out_word)
        
    return ' '.join(result)

In [60]:
generated = generate_seq(model, word_to_index, seq_length, 50)
print(generated)

Write the beginning of your poem, the Drake machine will complete it. Your input is: sdahfjashdfkjashd asdfh


Here is your poem: 

['coke', 'cookin', "problem's", 'headache', 'fifth', 'ovulating', 'birth', 'dear', 'neither', 'grips', 'aight', 'pacquiao', 'truthful', 'mann', 'babe', 'misconducts', 'extraterrestrial', 'dodgers', 'smack', 'ruin', 'jade', 'chasing', 'cars', 'poppins', 'tear', 'legendary', 'sorry', 'plenty', 'lewinsky', 'outdo', 'raps', 'growing', 'wide', 'sister', "you'ont", 'opposites', 'winks', 'glide', 'huge', 'southpaw', 'grasp', 'symbolic', 'harris', 'come', 'helped', 'muster', 'sit', 'beer', 'sweden', 'sdahfjashdfkjashd', 'asdfh']
envision fellatio fellow fellatio fellow denver clothes flippity christmas flippity cat triangle uh sa minimum fellatio fellow fellatio fellow fellatio groups resentment lazy hangs pectations jump pigeonholed ho's pinkett clothes pinkett clothes pinkett clothes pinkett lazy hangs bury idaho west pigeonholed fellatio fellow fellatio fellow 